In [1]:
import numpy as np

In [2]:
np.random.seed(42)

In [33]:
x = (np.random.uniform(0,15,size=[992,2])).astype(np.float64)
y = (np.random.uniform(0,15,size=[992,2])).astype(np.float64)

In [34]:
x

array([[  9.11519742,  13.68309431],
       [  5.78094628,  10.7650165 ],
       [ 11.25902431,  10.79982617],
       ..., 
       [ 12.30108502,   4.4334878 ],
       [ 10.70330149,   5.43787107],
       [  1.73501205,  13.70905166]])

In [37]:
992/8

124.0

In [44]:
x[0:4],x[5:9],x[10:14]
s=list()
def next_batch(x,step,batch_size):
    return x[batch_size*step:batch_size*step+batch_size]
for i in range(0,4):
    print(next_batch(x,i,124))


[[  9.11519742  13.68309431]
 [  5.78094628  10.7650165 ]
 [ 11.25902431  10.79982617]
 [ 13.16104716   9.65429038]
 [ 12.95163279   8.99337518]
 [  3.3656407   10.86451908]
 [  7.94555278   5.65699042]
 [  6.08476811   2.42230539]
 [  3.29278421  10.89234169]
 [  9.22330787   6.28677832]
 [ 13.71083104   3.01354429]
 [ 12.71662909   7.581098  ]
 [  3.75745599   5.59212734]
 [  5.22513853   4.79803735]
 [ 11.07788219   3.1545684 ]
 [  1.93753579   7.49722858]
 [  0.04269555   1.82394539]
 [  1.47018745  11.90573574]
 [  1.58900902  12.77878012]
 [  1.161669     6.20419098]
 [ 12.17826209   6.62001568]
 [ 12.19676086   5.82217843]
 [  2.58380561  10.53764229]
 [  6.05865888  10.14219394]
 [  3.77664924   2.65367475]
 [  3.41754702   7.93612655]
 [ 11.06670197   3.74096867]
 [  0.99654691  10.24855381]
 [  3.98160029  11.42124382]
 [  3.61313838  10.76233567]
 [  6.87429219   6.03772335]
 [  9.80370124   7.4608059 ]
 [ 13.22519951  14.91622905]
 [ 10.0647387   12.53001447]
 [  4.53394643

# Distancias entre vectores

* [Similitud coseno](http://reference.wolfram.com/language/ref/CosineDistance.html):
$$
CosineDistance[\vec{u},\vec{v}] = \frac{\vec{u}\cdot\vec{v}}{\lVert \vec{u} 
\lVert \lVert \vec{v} \lVert}
$$

---

* [Distancia euclidiana](http://reference.wolfram.com/language/ref/EuclideanDistance.html):
$$
EuclidianDistance[\vec{u},\vec{v}] = \sqrt{ \sum_{k=1}^n (u_i-v_i)^2}
$$

In [20]:
def get_distance(vector, matrix, distance="cos"):

    
    # Distancia coseno
    if distance == "cos":
        # Medir distancias
        tmp_dist = list(enumerate(((np.matmul(vector, matrix.T) /
                                        (np.linalg.norm(vector) *
                                         np.sqrt(np.einsum('ij,ij->i',
                                                     matrix, matrix,dtype=np.float64)))))))
        # Se ordena lista según distancias más cercanas.
        tmp_dist = sorted(tmp_dist, key=lambda dist: dist[1],reverse=True)
        distances = tmp_dist[:10]
        del tmp_dist
        # Retorna 10 vectores más cercanos.
        return distances

    # Distancia euclidiana
    tmp_dist = [(i, np.linalg.norm(vector - matrix[i]))
                for i in range(matrix.shape[0])]
    tmp_dist = sorted(tmp_dist, key=lambda dist: dist[1])
    distances = tmp_dist[:10]
    del tmp_dist
    return distances

In [25]:
import time
c = None
tic = time.time()
c = (((np.matmul(x[0], x.T) /
       (np.linalg.norm(x[0]) *
        np.sqrt(np.einsum('ij,ij->i',
                          x, matrix,dtype=np.float64))))))


toc = time.time()
#print(type(c))
print(c[:20])
print((toc-tic)*1000)

tic = 0
toc = 0
del c
tic = time.time()
c = [(((np.dot(x[0],x[i]))/(np.linalg.norm(x[0])*np.linalg.norm(x[i]))))
                for i in range(x.shape[0])]
toc = time.time()
#print(type(c))
print(c[:20])
print(1000*(toc-tic))

NameError: name 'vector' is not defined

In [22]:
dist = get_distance(x[0],x)

In [23]:
dist[:10]

[(0, 1.0000000000000004),
 (201, 0.75932451884122965),
 (674, 0.75919983780199474),
 (374, 0.75914880365300452),
 (758, 0.7583237462414032),
 (10, 0.75819602619028348),
 (112, 0.75815723518423495),
 (154, 0.75742587802360395),
 (982, 0.75727998824148901),
 (37, 0.75717784685635137)]

In [19]:
dist[:10]

[(0, -4.4408920985006262e-16),
 (201, 0.24067548115877035),
 (674, 0.24080016219800526),
 (374, 0.24085119634699548),
 (758, 0.2416762537585968),
 (10, 0.24180397380971652),
 (112, 0.24184276481576505),
 (154, 0.24257412197639605),
 (982, 0.24272001175851099),
 (37, 0.24282215314364863)]

In [ ]:
dist = get_distance(x[0],x)

In [ ]:
dist[:10]

In [ ]:
dist.sort(key=lambda tup: tup[1])

In [ ]:
dist[:10]